In [39]:

import pandas as pd


In [40]:
# ! pip uninstall vaex -y
# ! pip install "vaex[full]"

In [41]:
import vaex

# If you want to open an existing HDF5 file with vaex, use vaex.open:
vaex_df = vaex.open("C:/Users/hari9/Downloads/large_data_exported.hdf5")


# If you have a pandas DataFrame and want to convert it to vaex, use:
# vaex_df = vaex.from_pandas(df)
# vaex_df = vaex.from_pandas("large_data.hdf5")

In [42]:
type(vaex_df)

vaex.dataframe.DataFrameLocal

In [43]:
# vaex_df.export_hdf5(r'C:/Users/hari9/Downloads/large_data_exported.hdf5')
# vaex_df.export_hdf5('large_data.hdf5')
import h5py
with h5py.File(r"C:/Users/hari9/Downloads/test_direct.hdf5", "w") as f:
    f.create_dataset("dummy", data=[1, 2, 3])

In [44]:
vaex_df = vaex.open('large_data.hdf5')

In [45]:
vaex_df.head()

#,f1,f2,f3,f4,target
0,0.203936,-0.291191,-0.617251,-1.4613,10.7967
1,-1.19543,0.0692141,0.291861,1.97251,11.154
2,-2.56027,-0.827846,0.405551,-0.132588,-20.3427
3,-1.1312,0.252654,0.961175,1.23687,1.40764
4,0.858774,-0.461605,1.0955,-0.87158,-17.0431
5,0.63273,2.07437,-0.700978,1.36085,49.2754
6,-1.92771,0.155743,-0.394158,-1.3712,11.0902
7,-0.165462,-0.481113,-1.29813,1.75204,-41.1136
8,-1.31883,0.393728,-1.83311,-0.599728,1.87724
9,-0.671377,-0.172073,-0.75585,1.05745,-26.2623


In [46]:
vaex_df = vaex_df.shuffle()
df_train, df_test = vaex_df.ml.train_test_split(test_size=0.2)

d:\all\.venv\lib\site-packages\vaex\ml\__init__.py:28: UserWarning: Make sure the DataFrame is shuffled
  warnings.warn('Make sure the DataFrame is shuffled')


In [47]:
df_train.shape

(24000000, 5)

In [48]:
df_test.shape

(6000000, 5)

In [49]:
from vaex.ml.sklearn import IncrementalPredictor
from sklearn.linear_model import SGDRegressor

features = ['f1','f2','f3','f4']
target = 'target'
model = SGDRegressor()
vaex_model = IncrementalPredictor(features=features, target=target, model=model, batch_size=500000)

# Fit the model FIRST
vaex_model.fit(df=df_train)

# Then transform/predict
df_test = vaex_model.transform(df_test)

from sklearn.metrics import r2_score, mean_absolute_error
print(r2_score(df_test['target'].values, df_test['prediction'].values))
print(mean_absolute_error(df_test['target'].values, df_test['prediction'].values))

0.5912421852802107
15.967093560881423


In [50]:
df_test = vaex_model.transform(df_test)

In [51]:
df_test.head()

#,f1,f2,f3,f4,target,prediction
0,0.276644,-0.0668065,0.340587,0.624418,-25.8545,-1.78298
1,-0.434699,-0.365077,-0.378762,0.803824,-7.42881,-9.40138
2,-0.443189,0.0784667,0.977384,1.83799,14.3699,1.35655
3,0.726944,-0.819236,-0.0308231,0.307952,-46.0164,-20.0165
4,1.46078,-2.07596,0.292618,-1.52276,-70.0876,-49.8833
5,0.514574,-0.255039,1.46753,0.125535,-25.2681,-5.94962
6,-0.474701,-0.085891,0.562383,-0.178928,-23.6089,-2.10914
7,0.739353,0.386559,-0.730177,-0.888132,14.7822,9.63518
8,0.0939356,-0.125148,-1.33577,0.351722,2.29813,-3.48511
9,-1.39615,-0.619555,-0.880879,0.695927,-3.79376,-15.871
